<a href="https://colab.research.google.com/github/sejin-sim/Math/blob/main/5_3_%EC%84%A0%ED%98%95%EA%B3%84%ED%9A%8D%EB%B2%95_%EB%AC%B8%EC%A0%9C%EC%99%80_%EC%9D%B4%EC%B0%A8%EA%B3%84%ED%9A%8D%EB%B2%95_%EB%AC%B8%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### 기본 셋팅

In [ ]:
import numpy as np

# %matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
mpl.rcParams['axes.unicode_minus'] = False
mpl.rcParams.update({"axes.grid" : True})

plt.rcParams["figure.figsize"] = (15,9)
plt.rc("font", size=18)  # 그림의 폰트 크기를 18로 고정

np.random.seed(42)

# %config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.


### 1) 선형계획법 문제

- 기본형 선형계획법(Linear Programming) : 방정식이나 부등식 제한 조건을 가지는 선형 모형(linear model)의 값을 최소화하는 문제 
> $$
\begin{align}
\text{목적함수 }
\arg\min_x c^Tx 
\end{align}
$$
<br>
> $$
\begin{align}
\text{제한조건_선형 연립방정식 }
Ax = b 
\end{align}
$$
<br>
> $$
\begin{align}
\text{부등식 제한조건_변수값이 모두 음수가 아닌 }
x \geq 0
\end{align}
$$

- 정규형(canonical form) 선형계획법 예시 : 부등식 조건을 허용
> $$
\begin{align}
\arg\min_x c^Tx 
\end{align}
$$
<br>
> $$
\begin{align}
Ax \leq b 
\end{align}
$$
<br>
> $$
\begin{align}
x \geq 0
\end{align}
$$


#### 예제

어떤 공장에서 두가지 제품을 생산해야 한다고 하자. 

* 제품 A와 제품 B 각각 100개 이상 생산해야 한다.
* 시간은 500시간 밖에 없다.
* 제품 A는 생산하는데 1시간이 걸리고 제품 B는 2시간이 걸린다.
* 특정 부품이 9800개밖에 없다.
* 제품 A는 생산하는데 특정 부품을 4개 필요로 하고 제품 B는 생산하는데 특정 부품을 5개 필요로 한다. 
* 제품 A의 이익은 하나당 3만원이고 제품 B의 이익은 하나당 5만원이다.


* 제품 A와 제품 B의 생산량을 각각 $x_1, x_2$라고 하면,
> $$ 
\begin{align}
\text{목적함수 } -3x_1 -5x_2 
\end{align}
$$
<br>
> $$ 
\begin{align}
\begin{aligned}
\text{제한조건 }
-x_1 & & &\leq& -100 \\
 & & -x_2 &\leq& -100 \\
x_1 &+& 2 x_2 &\leq& 500 \\
4x_1 &+& 5 x_2 &\leq& 9800 \\
\end{aligned}
\end{align}
$$
<br>
> $$ 
\begin{align}
\text{제한조건 }
x_1 \geq 0, \;\; x_2 \geq 0 
\end{align}
$$

- 정규형(=기본형) 선형계획법 문제로 표현 시
> $$
\begin{align}
\min_x 
\begin{bmatrix} -3 & -5 \end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 
\end{bmatrix}
\end{align}
$$
<br>
> $$ 
\begin{align}
\begin{bmatrix}
-1 & 0 \\
0 & -1 \\
1 & 2 \\
4 & 5 \\
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 
\end{bmatrix} \leq
\begin{bmatrix}
-100 \\ -100 \\ 500 \\ 9800
\end{bmatrix}
\end{align}
$$
<br>
> $$ 
\begin{align}
\begin{bmatrix}
x_1 \\ x_2
\end{bmatrix}\geq
\begin{bmatrix}
0 \\ 0
\end{bmatrix}
\end{align}
$$

### 2) 사이파이를 이용한 선형계획법 문제 계산

scipy.optimize 패키지의 `linprog()` 명령을 사용하면 선형계획법 문제를 풀 수 있다. 사용법은 다음과 같다.

```
linprog(c, A, b)
```

* `c`: 목적함수의 계수 벡터
* `A`: 등식 제한조건의 계수 행렬
* `b`: 등식 제한조건의 상수 벡터


#### 예제 : 앞선 예제를 코드로 계산
+ 제품 A를 300개, 제품 B를 100개 생산할 때 이익이 1400으로 최대가 됨을 알 수 있다.

In [ ]:
import scipy as sp 

A = np.array([[-1, 0], [0, -1], [1, 2], [4, 5]])
b = np.array([-100, -100, 500, 9800])
c = np.array([-3, -5])

result = sp.optimize.linprog(c, A, b)
result

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


     con: array([], dtype=float64)
     fun: -1399.9999948073819
 message: 'Optimization terminated successfully.'
     nit: 6
   slack: array([1.99999991e+02, 4.19920389e-06, 3.31138096e-07, 8.10000001e+03])
  status: 0
 success: True
       x: array([299.99999127, 100.0000042 ])

### 3) CVXPY를 이용한 선형계획법 문제 계산

CVXPY 또는 PuLP와 같은 파이썬 패키지를 사용하면 선형계획법 문제의 계수 행렬 $A$, $b$, $c$를 직접 숫자로 정의하지 않고 심볼로 정의하여 더 직관적인 파이썬 코드를 만들 수 있다. 다음 코드는 위에서 풀었던 예제를 CVXPY로 다시 계산한 것이다. 다만 이 방법은 변수나 조건의 수가 아주 많을 경우에는 심볼릭 연산으로 인해 속도가 느려질 수 있다.

CVXPY는 conda 패키지 매니저로 설치할 수 있다.

```
conda install cvxpy
```

In [ ]:
import cvxpy as cp

# 변수의 정의
a = cp.Variable()  # A의 생산량
b = cp.Variable()  # B의 생산량

# 조건의 정의
constraints = [
    a >= 100,  # A를 100개 이상 생산해야 한다.
    b >= 100,  # B를 100개 이상 생산해야 한다. 
    a + 2 * b <= 500, # 500시간 내에 생산해야 한다.
    4 * a + 5 * b <= 9800,  # 부품이 9800개 밖에 없다.
]

# 문제의 정의
obj = cp.Maximize(3 * a + 5 * b) # 제품 A의 이익은 하나당 3만원, B는 5만원
prob = cp.Problem(obj, constraints)

# 계산
prob.solve() 

# 결과
print("상태:", prob.status)
print("최적값:", a.value, b.value)

상태: optimal
최적값: 299.99999911572195 100.00000058337798


### 4) 이차계획법 문제 (QP 문제)

- 이차계획법(Quadratic Programming) 문제 : 방정식이나 부등식 제한 조건을 가지는 일반화된 이차형식(quadratic form)의 값을 최소화하는 문제
> $$
\begin{align}
\text{목적함수 } \dfrac{1}{2}x^TQx + c^Tx
\end{align}
$$
<br>
> $$
\begin{align}
\begin{aligned}
\text{등식 제한조건 }Ax = b \\
\text{부호 제한조건 }x \geq 0
\end{aligned}
\end{align}
$$
- 잔차 제곱합을 최소화하는 예측 모형에 추가적인 제한조건이 있으면 이차계획법 문제가 됨

#### 예제

- 앞 절에서 풀었던 등식 제한조건이 있는 최적화 문제도 사실은 이차계획법 문제다.

$$
\begin{align}
\arg\min_x x_1^2 + x_2^2
\end{align}
$$

$$
\begin{align}
x_1 + x_2 - 1 = 0
\end{align}
$$

- QP 형식으로 바꾸면 다음과 같다.

$$ 
\begin{align}
\arg\min_x
\dfrac{1}{2}
\begin{bmatrix}
x_1 & x_2
\end{bmatrix}
\begin{bmatrix}
2 & 0 \\ 0 & 2
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2
\end{bmatrix}
+ 
\begin{bmatrix}
0 & 0
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2
\end{bmatrix}
\end{align}
$$

$$
\begin{align}
\begin{bmatrix}
1 & 1
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2
\end{bmatrix} = 1
\end{align}
$$

### 5) CvxOpt를 이용한 이차계획법 문제 계산

CvxOpt라는 패키지를 사용하면 이차계획법 문제를 풀 수 있다. CvxOpt를 쓸 때는 NumPy의 `ndarray` 배열을 CvxOpt 전용의 `matrix` 자료형으로 바꿔야 한다. 또 정수 자료형을 사용하지 못하므로 항상 부동소수점 실수가 되도록 명시해야 한다. 

CvxOpt도 conda 패키지 매니저로 설치할 수 있다.

```
conda install cvxopt
```

In [ ]:
from cvxopt import matrix, solvers

Q = matrix(np.diag([2.0, 2.0]))
c = matrix(np.array([0.0, 0.0]))
A = matrix(np.array([[1.0, 1.0]]))
b = matrix(np.array([[1.0]]))

sol = solvers.qp(Q, c, A=A, b=b)
np.array(sol['x'])

array([[0.5],
       [0.5]])

#### 연습 문제 5.3.1

다음 문제가 QP 문제임을 보이고 $N=3$인 경우에 대해 QP 문제의 $Q$, $c$, $A$, $b$를 각각 구하라(문제에서 $x$는 벡터이고 $y$는 실수다).

$$
\begin{align}
\arg\min_{a_i} \left( \sum_{i=1}^N a_i - \dfrac{1}{2}\sum_{i=1}^N\sum_{j=1}^N a_i a_j y_i y_j x_i^T x_j \right)
\end{align}
$$

$$
\begin{align}
\sum_{i=1}^N a_i y_i = 0
\end{align}
$$

$$
\begin{align}
a_i \geq 0 
\end{align}
$$

- 답

$$ 
\begin{align}
\arg\min_x
-\dfrac{1}{2}
\begin{bmatrix}
a_1 & a_2 & a_3
\end{bmatrix}
\begin{bmatrix}
y_1 y_1 x_1^T x_1 & y_1 y_2 x_1^T x_2 & y_1 y_3 x_1^T x_3 \\
y_2 y_1 x_2^T x_1 & y_2 y_2 x_2^T x_2 & y_2 y_3 x_2^T x_3 \\
y_3 y_1 x_3^T x_1 & y_3 y_2 x_3^T x_2 & y_3 y_3 x_3^T x_3
\end{bmatrix}
\begin{bmatrix}
a_1 \\ a_2 \\ a_3
\end{bmatrix}
+ 
\begin{bmatrix}
1 & 1 & 1
\end{bmatrix}
\begin{bmatrix}
a_1 \\ a_2 \\ a_3
\end{bmatrix}
\end{align}
$$

$$
\begin{align}
\begin{bmatrix}
y_1 & y_2 & y_3
\end{bmatrix}
\begin{bmatrix}
a_1 \\ a_2 \\ a_3
\end{bmatrix} = 0 \\
\end{align}
$$

$$
\begin{align}
a_i \geq 0 
\end{align}
$$

$$
\begin{align}
Q = \begin{bmatrix}
y_1 y_1 x_1^T x_1 & y_1 y_2 x_1^T x_2 & y_1 y_3 x_1^T x_3 \\
y_2 y_1 x_2^T x_1 & y_2 y_2 x_2^T x_2 & y_2 y_3 x_2^T x_3 \\
y_3 y_1 x_3^T x_1 & y_3 y_2 x_3^T x_2 & y_3 y_3 x_3^T x_3
\end{bmatrix}
\end{align}
$$

$$
\begin{align}
c = \begin{bmatrix}
1 & 1 & 1
\end{bmatrix}
\end{align}
$$

$$
\begin{align}
A = \begin{bmatrix}
y_1 & y_2 & y_3
\end{bmatrix}
\end{align}
$$

$$
\begin{align}
B = 0
\end{align}
$$